Для начала нужно подключить библиотеки для работы с изображением

In [1]:
from PIL import Image
import random
import os

Далее создаем переменные rand_folder и folder. Так как у нас 7 папок, то и диапозон будет соответсвующий. Мы создаем список images по указаному пути, где находяться изображения

In [2]:
rand_folder = random.randint(1, 7)
folder = f'../data/DATASET/train/{rand_folder}'
images = [f for f in os.listdir(folder)
          if f.lower().endswith(('.jpg'))]

Далее нам нужно выбрать изображение случайным образом и написать полный путь к изображению

In [3]:
random_image = random.choice(images)
image_path = os.path.join(folder, random_image)

После создаем переменную photo и photo_data. В photo открываем само изображение, а в photo_data предмтавляем цвета в численное представление. В данном случае это будет список каждого пикселя в формате RGB.

In [4]:
photo = Image.open(image_path)
photo_data = list(photo.getdata())
print(image_path)
print(photo.size)
print(photo.mode)
print(photo_data)

../data/DATASET/train/6/train_07936_aligned.jpg
(100, 100)
RGB
[(64, 64, 64), (80, 80, 80), (52, 52, 52), (34, 34, 34), (77, 77, 77), (117, 117, 117), (67, 67, 67), (80, 80, 80), (132, 132, 132), (134, 134, 134), (143, 143, 143), (58, 58, 58), (80, 80, 80), (106, 106, 106), (99, 99, 99), (50, 50, 50), (75, 75, 75), (80, 80, 80), (90, 90, 90), (102, 102, 102), (106, 106, 106), (106, 106, 106), (110, 110, 110), (116, 116, 116), (121, 121, 121), (97, 97, 97), (66, 66, 66), (23, 23, 23), (24, 24, 24), (3, 3, 3), (3, 3, 3), (0, 0, 0), (4, 4, 4), (0, 0, 0), (0, 0, 0), (20, 20, 20), (6, 6, 6), (0, 0, 0), (15, 15, 15), (0, 0, 0), (0, 0, 0), (1, 1, 1), (0, 0, 0), (3, 3, 3), (8, 8, 8), (1, 1, 1), (0, 0, 0), (7, 7, 7), (3, 3, 3), (9, 9, 9), (0, 0, 0), (8, 8, 8), (0, 0, 0), (9, 9, 9), (1, 1, 1), (4, 4, 4), (2, 2, 2), (0, 0, 0), (8, 8, 8), (0, 0, 0), (6, 6, 6), (21, 21, 21), (30, 30, 30), (76, 76, 76), (134, 134, 134), (89, 89, 89), (84, 84, 84), (83, 83, 83), (38, 38, 38), (11, 11, 11), (15, 15, 1

Ответы на вопросы по первому этапу:
1. Режим изображений RGB, размер изображений 100х100. Все изображения в датасете имеют одинаковый размер
2. Изображение численно представленно в виде списка, состоящего из 10000 элементов(т.к. размер изображения 100х100). Каналы - это отдельные компоненты цвета, из котрых сосотоит полное изображения. В исходном изображение 3 канала типа RGB.
3. Пиксели имееют тип данных кортеж. В кортежележит три числа, они принимают значения от 0 до 255, например: (167, 144, 113)

Этап 2:
Нужно подключить библиотку torchvision.transforms

In [5]:
import torch
import torchvision.transforms as t

Далее мы создаем переменную pipline, где используем Grayscale, Resize,ToTensor и Normalize

In [ ]:
pipline = t.Compose([
    t.Grayscale(num_output_channels=1),
    t.Resize((100, 100)),
    t.ToTensor(),
    t.Normalize(mean=[0.5], std=[0.5])
])

После создаем переменную transofrm_image, где в наш конфеер кладём не трансформированное фото

In [ ]:
transform_image = pipline(photo)
print(transform_image)

tensor([[[-0.4980, -0.3725, -0.5922,  ...,  0.0196,  0.1529, -0.0824],
         [-0.1216, -0.1137, -0.7882,  ..., -0.7725, -0.4353,  0.3725],
         [-0.3098, -0.2784, -0.6941,  ..., -0.7961, -0.4118,  0.4353],
         ...,
         [-0.7804, -0.8824, -1.0000,  ...,  0.9451,  0.6706,  0.5843],
         [-0.4431, -0.6235, -0.8980,  ...,  0.9294,  0.6157,  0.4745],
         [-0.1765, -0.3647, -0.7020,  ...,  0.8980,  0.5373,  0.3176]]])


Ответы на вопросы:
1. Будет полезно избавиться от цветов, т.к. нам нужна геометрия лица, модель будет сфокусирована именно на объекте. Численное пердстваление изменилось, теперь у нас только 1 канал.
2. Нужно в батче иметь одинаковый размер, т.к. модель даст лучшие результаты, если все фотографии будут 1 размера
3. Трансформация преобразует все в тензор типа float32. c диапазоном данных [0, 1]. а также получили массив
4. Нормализация нужна для ускорения обучения, помощи к градиентному спуску, а также для повышения тончости. ДЛя черно-белого изображения стоит писать в mean и std [0.5]